In [14]:
import pandas as pd
import numpy as np
import os
#import spacy
from IPython.display import display
from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.models import KeyedVectors
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier


!pip install pysentiment2

import pysentiment2 as ps 

In [15]:
def calculate_polarity_subjectivity(df):
    lm = ps.LM()
    hiv4 = ps.HIV4()
    polarity_array = []
    subjectivity_array = []
    hiv_polarity = []
    hiv_subjectivity = []
    count=0
    count1=0
    for x in range(len(df['filteredtext'])):
        tokens_m = lm.tokenize(df['filteredtext'][x])
        score_m = lm.get_score(tokens_m)
        polarity_array.append(score_m['Polarity'])
        subjectivity_array.append(score_m['Subjectivity'])
        tokens_hiv = hiv4.tokenize(df['filteredtext'][x])
        score_hiv = hiv4.get_score(tokens_hiv)
        hiv_polarity.append(score_hiv['Polarity'])
        hiv_subjectivity.append(score_hiv['Subjectivity'])
        if score_m['Polarity']*score_hiv['Polarity']<0:
            count+=1
    feature_df = pd.DataFrame()
    feature_df['Mcdonald_Polarity'] = polarity_array
    feature_df['Mcdonald_Subjectivity'] = subjectivity_array
    feature_df['HIV_Polarity'] = hiv_polarity
    feature_df['HIV_Subjectivity'] = hiv_subjectivity
    return feature_df

In [16]:
def get_glove_embeddings(df):
        word_list = []
        for i in df['filteredtext']:
            x = i[1:-1].split(", ")
            words = []
            for j in x:
                s = j.split(" ")
                for k in s:
                    words.append(k)
            word_list.append(words)
        filename = './data/glove.6B.100d.txt.word2vec'
        model = KeyedVectors.load_word2vec_format(filename, binary=False)


        embedding_list = []
        for i in word_list:
            embeddings = []
            for j in i:
                try:
                    glov = model[j]
                    embeddings.append(glov)
                except:
                    continue
            embedding_list.append(embeddings)
        return embedding_list

In [17]:
def tfIDFvectorization(input_df):
    vectorizer = TfidfVectorizer()
    vectors = vectorizer.fit_transform(list(input_df["filteredtext"]))
    feature_names = vectorizer.get_feature_names()
    dense = vectors.todense()
    denselist = dense.tolist()
    df = pd.DataFrame(denselist, columns=feature_names)
    normalize(df)
    return df

In [32]:
def logisticRegression():
    clf = LogisticRegression(random_state=10)
    clf.fit(X_train, y_train)
    # print(classification_report(y_test, clf.predict(X_test)))
    # print("LR Training Score: ",clf.score(X_train,y_train))
    # print("LR F1 Score:",f1_score(y_test,clf.predict(X_test),zero_division=0))
    # print("LR Accuracy:",accuracy_score(y_test,clf.predict(X_test)))
    return accuracy_score(y_test,clf.predict(X_test))

In [19]:
def naiveBayes():
    clf_NB = GaussianNB()
    clf_NB.fit(X_train,y_train)
    # print(classification_report(y_test, clf_NB.predict(X_test)))
    # print("NB Training Score:",clf_NB.score(X_train,y_train))
    # print("NB F1 Score: ",f1_score(y_test,clf_NB.predict(X_test),zero_division=0))
    # print("NB Accuracy:",accuracy_score(y_test,clf_NB.predict(X_test)))
    return accuracy_score(y_test,clf_NB.predict(X_test))

In [20]:
def mlp():
    clf = MLPClassifier()
    clf.fit(X_train, y_train)
    # print(classification_report(y_test, clf.predict(X_test)))
    # print("MLP Training Score: ",clf.score(X_train,y_train))
    # print("MLP F1 Score:",f1_score(y_test,clf.predict(X_test),zero_division=0))
    # print("MLP Accuracy:",accuracy_score(y_test,clf.predict(X_test)))
    return accuracy_score(y_test,clf.predict(X_test))

In [21]:

def linearSVM():
    clf = SVC(kernel="linear")
    clf.fit(X_train, y_train)
    # print(classification_report(y_test, clf.predict(X_test)))
    # print("Linear SVM Training Score: ",clf.score(X_train,y_train))
    # print("Linear SVM F1 Score:",f1_score(y_test,clf.predict(X_test),zero_division=0))
    # print("Linear Accuracy:",accuracy_score(y_test,clf.predict(X_test)))
    return accuracy_score(y_test,clf.predict(X_test))
    

In [22]:
def randomForest():
    clf = RandomForestClassifier(n_estimators=1000)
    clf.fit(X_train, y_train)
    # print(classification_report(y_test, clf.predict(X_test)))
    # print("RF Training Score: ",clf.score(X_train,y_train))
    # print("RF F1 Score:",f1_score(y_test,clf.predict(X_test),zero_division=0))
    # print("RF Accuracy:",accuracy_score(y_test,clf.predict(X_test)))
    return accuracy_score(y_test,clf.predict(X_test))
    

In [23]:
amazon60 = pd.read_csv("data/beforeafter/amazon60.csv")
tfidf_df = tfIDFvectorization(amazon60)
polsub_df = calculate_polarity_subjectivity(amazon60)
#glove_df=get_glove_embeddings(amazon5)
print(tfidf_df.shape)
print(polsub_df.shape)


(2955, 6087)
(2955, 4)


In [ ]:
glove_df=get_glove_embeddings(amazon5)

In [ ]:
glove_df = pd.Dataframe(glove_df, columns=["Glove"])

In [30]:
feature_df = pd.concat([tfidf_df, polsub_df],axis=1)
#feature_df=pd.concat([feature_df,glove_df],axis=1)
print(feature_df.shape)


(2955, 6091)


In [31]:


pca = PCA(n_components=2955)
amazon60_tfidf_reduced = pca.fit_transform(feature_df)
labels = amazon60["label"]
X_train, X_test, y_train, y_test = train_test_split(amazon60_tfidf_reduced, labels, test_size=0.2, random_state=42)
# if temp>acc1:
#     acc1=temp
#     n1=n

print(logisticRegression())

print(naiveBayes())

print(mlp())

print(linearSVM())
print(randomForest())

0.48223350253807107
0.47884940778341795
0.5177664974619289
0.4890016920473773
0.5363790186125211


In [33]:
print(logisticRegression())

0.48223350253807107


In [35]:
!pip install xgboost

     |████████████████████████████████| 1.2 MB 4.2 MB/s 


In [37]:
from xgboost import XGBClassifier
from sklearn import preprocessing
# scale the data
minmaxScaler = preprocessing.MinMaxScaler()
scaled_x_train = minmaxScaler.fit_transform(X_train)
scaled_x_test = minmaxScaler.fit_transform(X_test)
# create and train SVM model
xgboost_model = XGBClassifier()
xgboost_model.fit(scaled_x_train, y_train)
# generate prediction
y_train_pred = xgboost_model.predict(scaled_x_train)
y_test_pred = xgboost_model.predict(scaled_x_test)
# display train and test accuracy
train_accuracy = accuracy_score(y_train, y_train_pred)
print('Training accuracy is {0}'.format(train_accuracy))
test_accuracy = accuracy_score(y_test, y_test_pred)
print('Testing accuracy is {0}'.format(test_accuracy))

[18:49:46] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Training accuracy is 1.0
Testing accuracy is 0.5076142131979695


In [40]:
from sklearn import svm
# create and train SVM model
svm_model = svm.NuSVC(gamma='scale',probability=True)
svm_model.fit(scaled_x_train, y_train)
# generate prediction
y_train_pred = svm_model.predict(scaled_x_train)
y_test_pred = svm_model.predict(scaled_x_test)
# display train and test accuracy
train_accuracy = accuracy_score(y_train, y_train_pred)
print('Training accuracy is {0}'.format(train_accuracy))
test_accuracy = accuracy_score(y_test, y_test_pred)
print('Testing accuracy is {0}'.format(test_accuracy))

Training accuracy is 0.9991539763113367
Testing accuracy is 0.5194585448392555


In [45]:
#X_train, X_test, y_train, y_test
# Decision tree 
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier

decision_tree = tree.DecisionTreeClassifier()
decision_tree = decision_tree.fit(X_train, y_train)
res_pred = decision_tree.predict(X_test)
score = accuracy_score(y_test, res_pred)

print(score)


0.5093062605752962


In [ ]:
!pip install tensorflow==1.14.0
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM,Bidirectional
from keras.optimizers import Adam
from keras.layers import Embedding
model = Sequential()
# e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=1500, trainable=False)
# model.add(e)
model.add(Bidirectional(LSTM(128,input_shape=(1,X_train.shape[1]),dropout=0.2)))
model.add(Dense(2,activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.01),metrics=['accuracy'])
model.fit(trainX, y_train, epochs=30, batch_size=32, verbose=1,validation_split=0.2)
score, acc = model.evaluate(trainX, y_train)
print('Test score:', score)
print('Test accuracy:', acc)
testX=np.reshape(X_test, (X_test.shape[0], 1, X_train.shape[1]))
score, acc = model.evaluate(testX, y_test)
print('Test score:', score)
print('Test accuracy:', acc)